In [1]:
# Dependencies
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2

In [2]:
# Read in 2019 NYC data
nyc_may1 = "NYC/may_2019.csv"
nyc_may1_df = pd.read_csv(nyc_may1)
nyc_may1_df

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,139,2019-05-01 00:00:01.9010,2019-05-01 00:02:21.5170,447,8 Ave & W 52 St,40.763707,-73.985162,423,W 54 St & 9 Ave,40.765849,-73.986905,31170,Subscriber,1983,1
1,754,2019-05-01 00:00:03.0210,2019-05-01 00:12:37.6920,3258,W 27 St & 10 Ave,40.750182,-74.002184,3255,8 Ave & W 31 St,40.750585,-73.994685,25560,Customer,1969,0
2,2308,2019-05-01 00:00:04.6270,2019-05-01 00:38:33.1710,3093,N 6 St & Bedford Ave,40.717452,-73.958509,3676,Van Brunt St & Van Dyke St,40.675833,-74.014726,33369,Subscriber,1978,1
3,143,2019-05-01 00:00:19.3340,2019-05-01 00:02:42.5200,3486,Schermerhorn St & Bond St,40.688417,-73.984517,3412,Pacific St & Nevins St,40.685376,-73.983021,32041,Subscriber,1997,1
4,138,2019-05-01 00:00:22.1840,2019-05-01 00:02:40.6480,388,W 26 St & 10 Ave,40.749718,-74.002950,494,W 26 St & 8 Ave,40.747348,-73.997236,35237,Subscriber,1967,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1924558,304,2019-05-31 23:59:45.5760,2019-06-01 00:04:50.4620,3175,W 70 St & Amsterdam Ave,40.777480,-73.982886,423,W 54 St & 9 Ave,40.765849,-73.986905,32285,Subscriber,1987,2
1924559,324,2019-05-31 23:59:46.4650,2019-06-01 00:05:11.1630,2003,1 Ave & E 18 St,40.733812,-73.980544,536,1 Ave & E 30 St,40.741444,-73.975361,33874,Subscriber,1994,2
1924560,1926,2019-05-31 23:59:51.5060,2019-06-01 00:31:58.3920,146,Hudson St & Reade St,40.716250,-74.009106,422,W 59 St & 10 Ave,40.770513,-73.988038,33395,Subscriber,1985,1
1924561,2047,2019-05-31 23:59:52.4170,2019-06-01 00:34:00.1970,3658,W 18 St & 9 Ave,40.743534,-74.003676,2008,Little West St & 1 Pl,40.705693,-74.016777,27624,Subscriber,1987,0


In [3]:
# Check May df info
nyc_may1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1924563 entries, 0 to 1924562
Data columns (total 15 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   tripduration             int64  
 1   starttime                object 
 2   stoptime                 object 
 3   start station id         int64  
 4   start station name       object 
 5   start station latitude   float64
 6   start station longitude  float64
 7   end station id           int64  
 8   end station name         object 
 9   end station latitude     float64
 10  end station longitude    float64
 11  bikeid                   int64  
 12  usertype                 object 
 13  birth year               int64  
 14  gender                   int64  
dtypes: float64(4), int64(6), object(5)
memory usage: 220.2+ MB


In [4]:
nyc_may2 = "NYC/may_2020.csv"
nyc_may2_df = pd.read_csv(nyc_may2)
nyc_may2_df

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,336,2020-05-01 00:01:06.3380,2020-05-01 00:06:42.9560,2017,E 43 St & 2 Ave,40.750224,-73.971214,465,Broadway & W 41 St,40.755136,-73.986580,18015,Subscriber,1993,1
1,853,2020-05-01 00:01:08.1950,2020-05-01 00:15:21.2150,3376,E 65 St & 2 Ave,40.764719,-73.962221,478,11 Ave & W 41 St,40.760301,-73.998842,35905,Subscriber,1989,1
2,78,2020-05-01 00:01:08.5260,2020-05-01 00:02:27.0490,396,Lefferts Pl & Franklin Ave,40.680342,-73.955769,3789,Fulton St & Irving Pl,40.681860,-73.959432,33350,Subscriber,1938,2
3,1185,2020-05-01 00:02:22.4380,2020-05-01 00:22:08.1220,3630,Frederick Douglass Blvd & W 115 St,40.803865,-73.955931,3506,Lexington Ave & E 120 St,40.801307,-73.939817,43790,Subscriber,1996,1
4,272,2020-05-01 00:02:32.8280,2020-05-01 00:07:05.0880,3776,Central Ave & Starr Street,40.700003,-73.928340,3068,Humboldt St & Varet St,40.703172,-73.940636,39912,Subscriber,1993,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1487885,981,2020-05-31 23:59:19.0310,2020-06-01 00:15:40.5590,313,Washington Ave & Park Ave,40.696102,-73.967510,217,Old Fulton St,40.702772,-73.993836,43706,Subscriber,1972,2
1487886,367,2020-05-31 23:59:36.2270,2020-06-01 00:05:43.6200,545,E 23 St & 1 Ave,40.736502,-73.978095,501,FDR Drive & E 35 St,40.744219,-73.971212,41321,Subscriber,1994,2
1487887,1798,2020-05-31 23:59:37.7250,2020-06-01 00:29:36.6800,450,W 49 St & 8 Ave,40.762272,-73.987882,485,W 37 St & 5 Ave,40.750380,-73.983390,37418,Subscriber,1998,1
1487888,718,2020-05-31 23:59:47.7090,2020-06-01 00:11:46.5210,3691,28 Ave & 44 St,40.764089,-73.910651,3691,28 Ave & 44 St,40.764089,-73.910651,41190,Customer,1969,0


In [5]:
nyc_may2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1487890 entries, 0 to 1487889
Data columns (total 15 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   tripduration             1487890 non-null  int64  
 1   starttime                1487890 non-null  object 
 2   stoptime                 1487890 non-null  object 
 3   start station id         1487890 non-null  int64  
 4   start station name       1487890 non-null  object 
 5   start station latitude   1487890 non-null  float64
 6   start station longitude  1487890 non-null  float64
 7   end station id           1487890 non-null  int64  
 8   end station name         1487890 non-null  object 
 9   end station latitude     1487890 non-null  float64
 10  end station longitude    1487890 non-null  float64
 11  bikeid                   1487890 non-null  int64  
 12  usertype                 1487890 non-null  object 
 13  birth year               1487890 non-null 

In [6]:
nyc_may_df = pd.concat([nyc_may1_df, nyc_may2_df],0)
nyc_may_df

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,139,2019-05-01 00:00:01.9010,2019-05-01 00:02:21.5170,447,8 Ave & W 52 St,40.763707,-73.985162,423,W 54 St & 9 Ave,40.765849,-73.986905,31170,Subscriber,1983,1
1,754,2019-05-01 00:00:03.0210,2019-05-01 00:12:37.6920,3258,W 27 St & 10 Ave,40.750182,-74.002184,3255,8 Ave & W 31 St,40.750585,-73.994685,25560,Customer,1969,0
2,2308,2019-05-01 00:00:04.6270,2019-05-01 00:38:33.1710,3093,N 6 St & Bedford Ave,40.717452,-73.958509,3676,Van Brunt St & Van Dyke St,40.675833,-74.014726,33369,Subscriber,1978,1
3,143,2019-05-01 00:00:19.3340,2019-05-01 00:02:42.5200,3486,Schermerhorn St & Bond St,40.688417,-73.984517,3412,Pacific St & Nevins St,40.685376,-73.983021,32041,Subscriber,1997,1
4,138,2019-05-01 00:00:22.1840,2019-05-01 00:02:40.6480,388,W 26 St & 10 Ave,40.749718,-74.002950,494,W 26 St & 8 Ave,40.747348,-73.997236,35237,Subscriber,1967,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1487885,981,2020-05-31 23:59:19.0310,2020-06-01 00:15:40.5590,313,Washington Ave & Park Ave,40.696102,-73.967510,217,Old Fulton St,40.702772,-73.993836,43706,Subscriber,1972,2
1487886,367,2020-05-31 23:59:36.2270,2020-06-01 00:05:43.6200,545,E 23 St & 1 Ave,40.736502,-73.978095,501,FDR Drive & E 35 St,40.744219,-73.971212,41321,Subscriber,1994,2
1487887,1798,2020-05-31 23:59:37.7250,2020-06-01 00:29:36.6800,450,W 49 St & 8 Ave,40.762272,-73.987882,485,W 37 St & 5 Ave,40.750380,-73.983390,37418,Subscriber,1998,1
1487888,718,2020-05-31 23:59:47.7090,2020-06-01 00:11:46.5210,3691,28 Ave & 44 St,40.764089,-73.910651,3691,28 Ave & 44 St,40.764089,-73.910651,41190,Customer,1969,0


In [7]:
nyc_may_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3412453 entries, 0 to 1487889
Data columns (total 15 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   tripduration             int64  
 1   starttime                object 
 2   stoptime                 object 
 3   start station id         int64  
 4   start station name       object 
 5   start station latitude   float64
 6   start station longitude  float64
 7   end station id           int64  
 8   end station name         object 
 9   end station latitude     float64
 10  end station longitude    float64
 11  bikeid                   int64  
 12  usertype                 object 
 13  birth year               int64  
 14  gender                   int64  
dtypes: float64(4), int64(6), object(5)
memory usage: 416.6+ MB


In [8]:
nyc_june1 = "NYC/june_2019.csv"
nyc_june1_df = pd.read_csv(nyc_june1)
nyc_june1_df

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,330,2019-06-01 00:00:01.5000,2019-06-01 00:05:31.7600,3602,31 Ave & 34 St,40.763154,-73.920827,3570,35 Ave & 37 St,40.755733,-73.923661,20348,Subscriber,1992,1
1,830,2019-06-01 00:00:04.2400,2019-06-01 00:13:55.1470,3054,Greene Ave & Throop Ave,40.689493,-73.942061,3781,Greene Av & Myrtle Av,40.698568,-73.918877,34007,Subscriber,1987,2
2,380,2019-06-01 00:00:06.0190,2019-06-01 00:06:26.7790,229,Great Jones St,40.727434,-73.993790,326,E 11 St & 1 Ave,40.729538,-73.984267,20587,Subscriber,1990,2
3,1155,2019-06-01 00:00:06.7760,2019-06-01 00:19:22.5380,3771,McKibbin St & Bogart St,40.706237,-73.933871,3016,Kent Ave & N 7 St,40.720368,-73.961651,33762,Subscriber,1987,1
4,1055,2019-06-01 00:00:07.5200,2019-06-01 00:17:42.5580,441,E 52 St & 2 Ave,40.756014,-73.967416,3159,W 67 St & Broadway,40.774925,-73.982666,31290,Subscriber,1973,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2125365,232,2019-06-30 23:59:32.8920,2019-07-01 00:03:25.6310,3113,Greenpoint Ave & Manhattan Ave,40.730260,-73.953940,3107,Bedford Ave & Nassau Ave,40.723117,-73.952123,25779,Subscriber,1972,1
2125366,477,2019-06-30 23:59:33.6990,2019-07-01 00:07:31.0440,445,E 10 St & Avenue A,40.727408,-73.981420,3737,Stanton St & Norfolk St,40.720747,-73.986274,39522,Subscriber,1981,2
2125367,300,2019-06-30 23:59:47.4050,2019-07-01 00:04:48.1850,3307,West End Ave & W 94 St,40.794165,-73.974124,3320,Central Park West & W 100 St,40.794067,-73.962868,26730,Subscriber,1981,1
2125368,906,2019-06-30 23:59:51.5980,2019-07-01 00:14:58.3210,402,Broadway & E 22 St,40.740343,-73.989551,400,Pitt St & Stanton St,40.719261,-73.981780,38286,Subscriber,1996,1


In [9]:
nyc_june1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2125370 entries, 0 to 2125369
Data columns (total 15 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   tripduration             int64  
 1   starttime                object 
 2   stoptime                 object 
 3   start station id         int64  
 4   start station name       object 
 5   start station latitude   float64
 6   start station longitude  float64
 7   end station id           int64  
 8   end station name         object 
 9   end station latitude     float64
 10  end station longitude    float64
 11  bikeid                   int64  
 12  usertype                 object 
 13  birth year               int64  
 14  gender                   int64  
dtypes: float64(4), int64(6), object(5)
memory usage: 243.2+ MB


In [10]:
nyc_june2 = "NYC/june_2020.csv"
nyc_june2_df = pd.read_csv(nyc_june2)
nyc_june2_df

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1062,2020-06-01 00:00:03.3720,2020-06-01 00:17:46.2080,3419,Douglass St & 4 Ave,40.679279,-73.981540,3419,Douglass St & 4 Ave,40.679279,-73.981540,39852,Customer,1997,2
1,3810,2020-06-01 00:00:03.5530,2020-06-01 01:03:33.9360,366,Clinton Ave & Myrtle Ave,40.693261,-73.968896,336,Sullivan St & Washington Sq,40.730477,-73.999061,37558,Subscriber,1969,0
2,1017,2020-06-01 00:00:09.6140,2020-06-01 00:17:06.8330,389,Broadway & Berry St,40.710446,-73.965251,3562,Classon Ave & St Marks Ave,40.676520,-73.959608,37512,Customer,1988,2
3,226,2020-06-01 00:00:12.1780,2020-06-01 00:03:58.8640,3255,8 Ave & W 31 St,40.750585,-73.994685,505,6 Ave & W 33 St,40.749013,-73.988484,39674,Customer,1969,0
4,1437,2020-06-01 00:00:21.2550,2020-06-01 00:24:18.9650,367,E 53 St & Lexington Ave,40.758281,-73.970694,497,E 17 St & Broadway,40.737050,-73.990093,21093,Customer,1997,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1882268,685,2020-06-30 23:59:41.1160,2020-07-01 00:11:06.7790,503,E 20 St & Park Ave,40.738274,-73.987520,3746,6 Ave & Broome St,40.724308,-74.004730,21056,Customer,1993,2
1882269,446,2020-06-30 23:59:46.4260,2020-07-01 00:07:13.0860,3699,W 50 St & 9 Ave,40.763605,-73.989180,523,W 38 St & 8 Ave,40.754666,-73.991382,30164,Customer,1969,0
1882270,439,2020-06-30 23:59:47.4770,2020-07-01 00:07:06.5590,3699,W 50 St & 9 Ave,40.763605,-73.989180,523,W 38 St & 8 Ave,40.754666,-73.991382,43742,Customer,1986,1
1882271,890,2020-06-30 23:59:53.3950,2020-07-01 00:14:43.4270,3852,Stewart Ave & Johnson Ave,40.708690,-73.925870,3054,Greene Ave & Throop Ave,40.689493,-73.942061,15787,Subscriber,1994,1


In [11]:
nyc_june2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1882273 entries, 0 to 1882272
Data columns (total 15 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   tripduration             int64  
 1   starttime                object 
 2   stoptime                 object 
 3   start station id         int64  
 4   start station name       object 
 5   start station latitude   float64
 6   start station longitude  float64
 7   end station id           int64  
 8   end station name         object 
 9   end station latitude     float64
 10  end station longitude    float64
 11  bikeid                   int64  
 12  usertype                 object 
 13  birth year               int64  
 14  gender                   int64  
dtypes: float64(4), int64(6), object(5)
memory usage: 215.4+ MB


In [12]:
nyc_june_df = pd.concat([nyc_june1_df, nyc_june2_df],0)
nyc_june_df

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,330,2019-06-01 00:00:01.5000,2019-06-01 00:05:31.7600,3602,31 Ave & 34 St,40.763154,-73.920827,3570,35 Ave & 37 St,40.755733,-73.923661,20348,Subscriber,1992,1
1,830,2019-06-01 00:00:04.2400,2019-06-01 00:13:55.1470,3054,Greene Ave & Throop Ave,40.689493,-73.942061,3781,Greene Av & Myrtle Av,40.698568,-73.918877,34007,Subscriber,1987,2
2,380,2019-06-01 00:00:06.0190,2019-06-01 00:06:26.7790,229,Great Jones St,40.727434,-73.993790,326,E 11 St & 1 Ave,40.729538,-73.984267,20587,Subscriber,1990,2
3,1155,2019-06-01 00:00:06.7760,2019-06-01 00:19:22.5380,3771,McKibbin St & Bogart St,40.706237,-73.933871,3016,Kent Ave & N 7 St,40.720368,-73.961651,33762,Subscriber,1987,1
4,1055,2019-06-01 00:00:07.5200,2019-06-01 00:17:42.5580,441,E 52 St & 2 Ave,40.756014,-73.967416,3159,W 67 St & Broadway,40.774925,-73.982666,31290,Subscriber,1973,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1882268,685,2020-06-30 23:59:41.1160,2020-07-01 00:11:06.7790,503,E 20 St & Park Ave,40.738274,-73.987520,3746,6 Ave & Broome St,40.724308,-74.004730,21056,Customer,1993,2
1882269,446,2020-06-30 23:59:46.4260,2020-07-01 00:07:13.0860,3699,W 50 St & 9 Ave,40.763605,-73.989180,523,W 38 St & 8 Ave,40.754666,-73.991382,30164,Customer,1969,0
1882270,439,2020-06-30 23:59:47.4770,2020-07-01 00:07:06.5590,3699,W 50 St & 9 Ave,40.763605,-73.989180,523,W 38 St & 8 Ave,40.754666,-73.991382,43742,Customer,1986,1
1882271,890,2020-06-30 23:59:53.3950,2020-07-01 00:14:43.4270,3852,Stewart Ave & Johnson Ave,40.708690,-73.925870,3054,Greene Ave & Throop Ave,40.689493,-73.942061,15787,Subscriber,1994,1


In [13]:
nyc_june_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4007643 entries, 0 to 1882272
Data columns (total 15 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   tripduration             int64  
 1   starttime                object 
 2   stoptime                 object 
 3   start station id         int64  
 4   start station name       object 
 5   start station latitude   float64
 6   start station longitude  float64
 7   end station id           int64  
 8   end station name         object 
 9   end station latitude     float64
 10  end station longitude    float64
 11  bikeid                   int64  
 12  usertype                 object 
 13  birth year               int64  
 14  gender                   int64  
dtypes: float64(4), int64(6), object(5)
memory usage: 489.2+ MB


In [14]:
nyc_july1 = "NYC/july_2019.csv"
nyc_july1_df = pd.read_csv(nyc_july1)
nyc_july1_df

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,897,2019-07-01 00:00:00.1320,2019-07-01 00:14:58.0040,493.0,W 45 St & 6 Ave,40.756800,-73.982912,454.0,E 51 St & 1 Ave,40.754557,-73.965930,18340,Subscriber,1966,1
1,267,2019-07-01 00:00:05.1780,2019-07-01 00:04:32.4500,3143.0,5 Ave & E 78 St,40.776321,-73.964274,3226.0,W 82 St & Central Park West,40.782750,-73.971370,21458,Customer,1996,1
2,2201,2019-07-01 00:00:05.2130,2019-07-01 00:36:46.7490,317.0,E 6 St & Avenue B,40.724537,-73.981854,3469.0,India St & West St,40.731814,-73.959950,39874,Subscriber,1986,1
3,1660,2019-07-01 00:00:08.6010,2019-07-01 00:27:48.8050,249.0,Harrison St & Hudson St,40.718710,-74.009001,369.0,Washington Pl & 6 Ave,40.732241,-74.000264,38865,Subscriber,1988,1
4,109,2019-07-01 00:00:12.1580,2019-07-01 00:02:01.5670,3552.0,W 113 St & Broadway,40.805973,-73.964928,3538.0,W 110 St & Amsterdam Ave,40.802692,-73.962950,30256,Subscriber,1997,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2181059,644,2019-07-31 23:59:45.2950,2019-08-01 00:10:29.8640,274.0,Lafayette Ave & Fort Greene Pl,40.686919,-73.976682,3409.0,Bergen St & Smith St,40.686744,-73.990632,19018,Subscriber,1964,1
2181060,589,2019-07-31 23:59:52.7170,2019-08-01 00:09:42.0870,3285.0,W 87 St & Amsterdam Ave,40.788390,-73.974700,3356.0,Amsterdam Ave & W 66 St,40.774667,-73.984706,15330,Subscriber,1997,1
2181061,1302,2019-07-31 23:59:52.8280,2019-08-01 00:21:35.1090,331.0,Pike St & Monroe St,40.711731,-73.991930,309.0,Murray St & West St,40.714979,-74.013012,39730,Subscriber,1966,2
2181062,353,2019-07-31 23:59:57.1130,2019-08-01 00:05:50.1350,3255.0,8 Ave & W 31 St,40.750585,-73.994685,482.0,W 15 St & 7 Ave,40.739355,-73.999318,30149,Subscriber,1994,1


In [15]:
nyc_july1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2181064 entries, 0 to 2181063
Data columns (total 15 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   tripduration             int64  
 1   starttime                object 
 2   stoptime                 object 
 3   start station id         float64
 4   start station name       object 
 5   start station latitude   float64
 6   start station longitude  float64
 7   end station id           float64
 8   end station name         object 
 9   end station latitude     float64
 10  end station longitude    float64
 11  bikeid                   int64  
 12  usertype                 object 
 13  birth year               int64  
 14  gender                   int64  
dtypes: float64(6), int64(4), object(5)
memory usage: 249.6+ MB


In [16]:
nyc_july2 = "NYC/july_2020.csv"
nyc_july2_df = pd.read_csv(nyc_july2)
nyc_july2_df

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,341,2020-07-01 00:00:01.3020,2020-07-01 00:05:42.5650,3463,E 16 St & Irving Pl,40.735367,-73.987974,2003,1 Ave & E 18 St,40.733812,-73.980544,33861,Subscriber,1986,1
1,321,2020-07-01 00:00:01.3620,2020-07-01 00:05:22.9490,3463,E 16 St & Irving Pl,40.735367,-73.987974,2003,1 Ave & E 18 St,40.733812,-73.980544,31233,Subscriber,1990,2
2,2710,2020-07-01 00:00:06.6290,2020-07-01 00:45:17.3410,426,West St & Chambers St,40.717548,-74.013221,212,W 16 St & The High Line,40.743349,-74.006818,40329,Customer,1969,0
3,2685,2020-07-01 00:00:13.0220,2020-07-01 00:44:58.8640,426,West St & Chambers St,40.717548,-74.013221,212,W 16 St & The High Line,40.743349,-74.006818,17567,Subscriber,1998,1
4,191,2020-07-01 00:00:13.4660,2020-07-01 00:03:24.6230,3615,44 Dr & 21 St,40.748000,-73.946093,3127,9 St & 44 Rd,40.749660,-73.952100,43421,Subscriber,1992,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2105803,877,2020-07-31 23:59:48.9250,2020-08-01 00:14:26.2540,217,Old Fulton St,40.702772,-73.993836,398,Atlantic Ave & Furman St,40.691652,-73.999979,44073,Customer,1969,0
2105804,839,2020-07-31 23:59:51.7740,2020-08-01 00:13:50.9750,4044,8 Ave & W 38 St,40.754610,-73.991770,3163,Central Park West & W 68 St,40.773407,-73.977825,41797,Customer,2002,1
2105805,1042,2020-07-31 23:59:56.1460,2020-08-01 00:17:18.5090,3723,Cadman Plaza E & Johnson St,40.695317,-73.990157,350,Clinton St & Grand St,40.715595,-73.987030,24808,Subscriber,1974,1
2105806,699,2020-07-31 23:59:57.4310,2020-08-01 00:11:36.5420,161,LaGuardia Pl & W 3 St,40.729170,-73.998102,504,1 Ave & E 16 St,40.732219,-73.981656,41486,Subscriber,1991,1


In [17]:
nyc_july2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2105808 entries, 0 to 2105807
Data columns (total 15 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   tripduration             int64  
 1   starttime                object 
 2   stoptime                 object 
 3   start station id         int64  
 4   start station name       object 
 5   start station latitude   float64
 6   start station longitude  float64
 7   end station id           int64  
 8   end station name         object 
 9   end station latitude     float64
 10  end station longitude    float64
 11  bikeid                   int64  
 12  usertype                 object 
 13  birth year               int64  
 14  gender                   int64  
dtypes: float64(4), int64(6), object(5)
memory usage: 241.0+ MB


In [18]:
nyc_july_df = pd.concat([nyc_july1_df, nyc_july2_df],0)
nyc_july_df

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,897,2019-07-01 00:00:00.1320,2019-07-01 00:14:58.0040,493.0,W 45 St & 6 Ave,40.756800,-73.982912,454.0,E 51 St & 1 Ave,40.754557,-73.965930,18340,Subscriber,1966,1
1,267,2019-07-01 00:00:05.1780,2019-07-01 00:04:32.4500,3143.0,5 Ave & E 78 St,40.776321,-73.964274,3226.0,W 82 St & Central Park West,40.782750,-73.971370,21458,Customer,1996,1
2,2201,2019-07-01 00:00:05.2130,2019-07-01 00:36:46.7490,317.0,E 6 St & Avenue B,40.724537,-73.981854,3469.0,India St & West St,40.731814,-73.959950,39874,Subscriber,1986,1
3,1660,2019-07-01 00:00:08.6010,2019-07-01 00:27:48.8050,249.0,Harrison St & Hudson St,40.718710,-74.009001,369.0,Washington Pl & 6 Ave,40.732241,-74.000264,38865,Subscriber,1988,1
4,109,2019-07-01 00:00:12.1580,2019-07-01 00:02:01.5670,3552.0,W 113 St & Broadway,40.805973,-73.964928,3538.0,W 110 St & Amsterdam Ave,40.802692,-73.962950,30256,Subscriber,1997,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2105803,877,2020-07-31 23:59:48.9250,2020-08-01 00:14:26.2540,217.0,Old Fulton St,40.702772,-73.993836,398.0,Atlantic Ave & Furman St,40.691652,-73.999979,44073,Customer,1969,0
2105804,839,2020-07-31 23:59:51.7740,2020-08-01 00:13:50.9750,4044.0,8 Ave & W 38 St,40.754610,-73.991770,3163.0,Central Park West & W 68 St,40.773407,-73.977825,41797,Customer,2002,1
2105805,1042,2020-07-31 23:59:56.1460,2020-08-01 00:17:18.5090,3723.0,Cadman Plaza E & Johnson St,40.695317,-73.990157,350.0,Clinton St & Grand St,40.715595,-73.987030,24808,Subscriber,1974,1
2105806,699,2020-07-31 23:59:57.4310,2020-08-01 00:11:36.5420,161.0,LaGuardia Pl & W 3 St,40.729170,-73.998102,504.0,1 Ave & E 16 St,40.732219,-73.981656,41486,Subscriber,1991,1


In [20]:
nyc_july_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4286872 entries, 0 to 2105807
Data columns (total 15 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   tripduration             int64  
 1   starttime                object 
 2   stoptime                 object 
 3   start station id         float64
 4   start station name       object 
 5   start station latitude   float64
 6   start station longitude  float64
 7   end station id           float64
 8   end station name         object 
 9   end station latitude     float64
 10  end station longitude    float64
 11  bikeid                   int64  
 12  usertype                 object 
 13  birth year               int64  
 14  gender                   int64  
dtypes: float64(6), int64(4), object(5)
memory usage: 523.3+ MB


In [21]:
nyc_aug1 = "NYC/aug_2019.csv"
nyc_aug1_df = pd.read_csv(nyc_aug1)
nyc_aug1_df

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,393,2019-08-01 00:00:01.4680,2019-08-01 00:06:35.3780,531.0,Forsyth St & Broome St,40.718939,-73.992663,408.0,Market St & Cherry St,40.710762,-73.994004,35305,Subscriber,1996,2
1,627,2019-08-01 00:00:01.9290,2019-08-01 00:10:29.7840,274.0,Lafayette Ave & Fort Greene Pl,40.686919,-73.976682,3409.0,Bergen St & Smith St,40.686744,-73.990632,38822,Subscriber,1998,2
2,1132,2019-08-01 00:00:04.0480,2019-08-01 00:18:56.1650,2000.0,Front St & Washington St,40.702551,-73.989402,3388.0,President St & Henry St,40.682800,-73.999904,18373,Subscriber,1988,1
3,1780,2019-08-01 00:00:04.1630,2019-08-01 00:29:44.7940,479.0,9 Ave & W 45 St,40.760193,-73.991255,473.0,Rivington St & Chrystie St,40.721101,-73.991925,25002,Subscriber,1988,1
4,1517,2019-08-01 00:00:05.4580,2019-08-01 00:25:23.4550,3312.0,1 Ave & E 94 St,40.781721,-73.945940,3312.0,1 Ave & E 94 St,40.781721,-73.945940,31198,Subscriber,1965,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2344219,216,2019-08-31 23:59:46.2930,2019-09-01 00:03:23.0360,116.0,W 17 St & 8 Ave,40.741776,-74.001497,509.0,9 Ave & W 22 St,40.745497,-74.001971,38655,Subscriber,1972,1
2344220,117,2019-08-31 23:59:47.7970,2019-09-01 00:01:45.7440,3429.0,Hanson Pl & Ashland Pl,40.685068,-73.977908,353.0,S Portland Ave & Hanson Pl,40.685396,-73.974315,25264,Subscriber,1982,2
2344221,1614,2019-08-31 23:59:48.1560,2019-09-01 00:26:42.7760,387.0,Centre St & Chambers St,40.712733,-74.004607,3440.0,Fulton St & Adams St,40.692418,-73.989495,28485,Customer,1969,0
2344222,1301,2019-08-31 23:59:58.3620,2019-09-01 00:21:39.7040,3168.0,Central Park West & W 85 St,40.784727,-73.969617,423.0,W 54 St & 9 Ave,40.765849,-73.986905,38664,Customer,1969,0


In [22]:
nyc_aug1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2344224 entries, 0 to 2344223
Data columns (total 15 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   tripduration             int64  
 1   starttime                object 
 2   stoptime                 object 
 3   start station id         float64
 4   start station name       object 
 5   start station latitude   float64
 6   start station longitude  float64
 7   end station id           float64
 8   end station name         object 
 9   end station latitude     float64
 10  end station longitude    float64
 11  bikeid                   int64  
 12  usertype                 object 
 13  birth year               int64  
 14  gender                   int64  
dtypes: float64(6), int64(4), object(5)
memory usage: 268.3+ MB


In [23]:
nyc_aug2 = "NYC/aug_2020.csv"
nyc_aug2_df = pd.read_csv(nyc_aug2)
nyc_aug2_df

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,2013,2020-08-01 00:00:00.8790,2020-08-01 00:33:34.7260,3295,Central Park W & W 96 St,40.791270,-73.964839,3992,W 169 St & Fort Washington Ave,40.842842,-73.942125,19706,Customer,1998,1
1,7002,2020-08-01 00:00:01.5910,2020-08-01 01:56:44.0720,3144,E 81 St & Park Ave,40.776777,-73.959010,3671,E 81 St & 2 Ave,40.774779,-73.954275,37191,Subscriber,1966,1
2,155,2020-08-01 00:00:03.5810,2020-08-01 00:02:38.6340,387,Centre St & Chambers St,40.712733,-74.004607,387,Centre St & Chambers St,40.712733,-74.004607,42561,Customer,1969,0
3,2720,2020-08-01 00:00:05.0130,2020-08-01 00:45:25.3270,3080,S 4 St & Rodney St,40.709340,-73.956080,3364,Carroll St & 5 Ave,40.675162,-73.981483,41607,Subscriber,1990,1
4,2938,2020-08-01 00:00:05.9420,2020-08-01 00:49:03.9640,3510,Adam Clayton Powell Blvd & W 123 St,40.807832,-73.949373,3282,5 Ave & E 88 St,40.783070,-73.959390,36366,Customer,1969,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2329509,1562,2020-08-31 23:59:29.7690,2020-09-01 00:25:31.9660,298,3 Ave & Schermerhorn St,40.686832,-73.979677,3423,West Drive & Prospect Park West,40.661063,-73.979453,37282,Customer,2001,1
2329510,1023,2020-08-31 23:59:32.1320,2020-09-01 00:16:35.3710,3508,St Nicholas Ave & Manhattan Ave,40.809725,-73.953149,3323,W 106 St & Central Park West,40.798186,-73.960591,35088,Customer,1993,2
2329511,600,2020-08-31 23:59:33.0920,2020-09-01 00:09:33.4670,320,Leonard St & Church St,40.717571,-74.005549,380,W 4 St & 7 Ave S,40.734011,-74.002939,36353,Subscriber,2002,1
2329512,737,2020-08-31 23:59:35.7900,2020-09-01 00:11:53.2310,3318,2 Ave & E 96 St,40.783964,-73.947167,3156,E 72 St & York Ave,40.766638,-73.953483,44935,Subscriber,1981,2


In [24]:
nyc_aug2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2329514 entries, 0 to 2329513
Data columns (total 15 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   tripduration             int64  
 1   starttime                object 
 2   stoptime                 object 
 3   start station id         int64  
 4   start station name       object 
 5   start station latitude   float64
 6   start station longitude  float64
 7   end station id           int64  
 8   end station name         object 
 9   end station latitude     float64
 10  end station longitude    float64
 11  bikeid                   int64  
 12  usertype                 object 
 13  birth year               int64  
 14  gender                   int64  
dtypes: float64(4), int64(6), object(5)
memory usage: 266.6+ MB


In [25]:
nyc_aug_df = pd.concat([nyc_aug1_df, nyc_aug2_df],0)
nyc_aug_df

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,393,2019-08-01 00:00:01.4680,2019-08-01 00:06:35.3780,531.0,Forsyth St & Broome St,40.718939,-73.992663,408.0,Market St & Cherry St,40.710762,-73.994004,35305,Subscriber,1996,2
1,627,2019-08-01 00:00:01.9290,2019-08-01 00:10:29.7840,274.0,Lafayette Ave & Fort Greene Pl,40.686919,-73.976682,3409.0,Bergen St & Smith St,40.686744,-73.990632,38822,Subscriber,1998,2
2,1132,2019-08-01 00:00:04.0480,2019-08-01 00:18:56.1650,2000.0,Front St & Washington St,40.702551,-73.989402,3388.0,President St & Henry St,40.682800,-73.999904,18373,Subscriber,1988,1
3,1780,2019-08-01 00:00:04.1630,2019-08-01 00:29:44.7940,479.0,9 Ave & W 45 St,40.760193,-73.991255,473.0,Rivington St & Chrystie St,40.721101,-73.991925,25002,Subscriber,1988,1
4,1517,2019-08-01 00:00:05.4580,2019-08-01 00:25:23.4550,3312.0,1 Ave & E 94 St,40.781721,-73.945940,3312.0,1 Ave & E 94 St,40.781721,-73.945940,31198,Subscriber,1965,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2329509,1562,2020-08-31 23:59:29.7690,2020-09-01 00:25:31.9660,298.0,3 Ave & Schermerhorn St,40.686832,-73.979677,3423.0,West Drive & Prospect Park West,40.661063,-73.979453,37282,Customer,2001,1
2329510,1023,2020-08-31 23:59:32.1320,2020-09-01 00:16:35.3710,3508.0,St Nicholas Ave & Manhattan Ave,40.809725,-73.953149,3323.0,W 106 St & Central Park West,40.798186,-73.960591,35088,Customer,1993,2
2329511,600,2020-08-31 23:59:33.0920,2020-09-01 00:09:33.4670,320.0,Leonard St & Church St,40.717571,-74.005549,380.0,W 4 St & 7 Ave S,40.734011,-74.002939,36353,Subscriber,2002,1
2329512,737,2020-08-31 23:59:35.7900,2020-09-01 00:11:53.2310,3318.0,2 Ave & E 96 St,40.783964,-73.947167,3156.0,E 72 St & York Ave,40.766638,-73.953483,44935,Subscriber,1981,2


In [26]:
nyc_aug_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4673738 entries, 0 to 2329513
Data columns (total 15 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   tripduration             int64  
 1   starttime                object 
 2   stoptime                 object 
 3   start station id         float64
 4   start station name       object 
 5   start station latitude   float64
 6   start station longitude  float64
 7   end station id           float64
 8   end station name         object 
 9   end station latitude     float64
 10  end station longitude    float64
 11  bikeid                   int64  
 12  usertype                 object 
 13  birth year               int64  
 14  gender                   int64  
dtypes: float64(6), int64(4), object(5)
memory usage: 570.5+ MB
